In [10]:
import os
import json
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")
NOTION_VERSION = os.getenv("NOTION_VERSION")

In [ ]:
class NotionAPI:
    def __init__(self, api_token: str):
        self.api_token = api_token
        self.base_url = "https://api.notion.com/v1"
        self.headers = {
            "Authorization": f"Bearer {api_token}",
            "Notion-Version": NOTION_VERSION,
            "Content-Type": "application/json",
        }

    # ---------- Databases ----------
    def get_database(self, database_id: str):
        url = f"{self.base_url}/databases/{database_id}"
        r = requests.get(url, headers=self.headers)
        return self._ok(r)

    def query_database(
        self,
        database_id: str,
        filter_obj=None,
        sorts=None,
        page_size: int = 100,
        start_cursor: str | None = None,
    ):
        """단일 요청(query) — next_cursor 반환"""
        url = f"{self.base_url}/databases/{database_id}/query"
        payload = {"page_size": page_size}
        if filter_obj:
            payload["filter"] = filter_obj
        if sorts:
            payload["sorts"] = sorts
        if start_cursor:
            payload["start_cursor"] = start_cursor

        r = requests.post(url, headers=self.headers, json=payload)
        return self._ok(r)

    def query_database_all(
        self, database_id: str, filter_obj=None, sorts=None, page_size: int = 100
    ):
        """페이지네이션 처리하여 DB의 모든 페이지 가져오기"""
        all_results = []
        cursor = None
        while True:
            data = self.query_database(
                database_id, filter_obj, sorts, page_size, cursor
            )
            all_results.extend(data["results"])
            if data.get("has_more"):
                cursor = data.get("next_cursor")
            else:
                break
        return all_results

    # ---------- Pages ----------
    def get_page(self, page_id: str):
        url = f"{self.base_url}/pages/{page_id}"
        r = requests.get(url, headers=self.headers)
        return self._ok(r)

    # ---------- Blocks (본문) ----------
    def get_block_children(
        self, block_id: str, start_cursor: str | None = None, page_size: int = 100
    ):
        url = f"{self.base_url}/blocks/{block_id}/children"
        params = {}
        if start_cursor:
            params["start_cursor"] = start_cursor
        params["page_size"] = page_size
        r = requests.get(url, headers=self.headers, params=params)
        return self._ok(r)

    def get_page_content_flat(self, page_id: str):
        """해당 페이지의 최상위 블록들을 페이지네이션으로 모두 모아 'flat' 리스트로 반환"""
        all_blocks = []
        cursor = None
        while True:
            data = self.get_block_children(page_id, start_cursor=cursor)
            all_blocks.extend(data["results"])
            if data.get("has_more"):
                cursor = data.get("next_cursor")
            else:
                break
        return all_blocks

    def get_block_tree(self, block_id: str):
        """
        block_id 하위의 블록들을 **재귀적으로** 모두 수집해 트리 형태로 반환.
        (페이지 전체 깊은 내용까지 가져오고 싶을 때 사용)
        """
        children = []
        cursor = None
        while True:
            data = self.get_block_children(block_id, start_cursor=cursor)
            for b in data["results"]:
                # 자식이 있는 블록이면 재귀적으로 children 채워 넣음
                if b.get("has_children"):
                    b["children"] = self.get_block_tree(b["id"])
                children.append(b)
            if data.get("has_more"):
                cursor = data.get("next_cursor")
            else:
                break
        return children

    # ---------- Utils ----------
    def _ok(self, response: requests.Response):
        if response.status_code // 100 == 2:
            return response.json()
        print(f"[Error] {response.status_code}")
        try:
            print(response.json())
        except Exception:
            print(response.text)
        raise RuntimeError("Notion API request failed")

    @staticmethod
    def block_to_text(block: dict) -> str:
        """대표적인 텍스트 블록들을 plain text로 변환 (필요 시 타입 추가)"""
        t = block.get("type")
        data = block.get(t, {})
        rich = data.get("rich_text", [])
        text = "".join([r.get("plain_text", "") for r in rich])

        if t in ("heading_1", "heading_2", "heading_3"):
            prefix = {"heading_1": "# ", "heading_2": "## ", "heading_3": "### "}[t]
            return prefix + text
        elif t == "to_do":
            checked = data.get("checked", False)
            return f"[{'x' if checked else ' '}] {text}"
        elif t in ("bulleted_list_item", "numbered_list_item"):
            bullet = "-" if t == "bulleted_list_item" else "1."
            return f"{bullet} {text}"
        elif t == "code":
            lang = data.get("language", "")
            code_text = "".join([r.get("plain_text", "") for r in rich])
            return f"```{lang}\n{code_text}\n```"
        else:
            return text  # paragraph 등 기본 처리


def render_blocks(blocks: list[dict], indent: int = 0):
    """트리 블록들을 콘솔에 예쁘게 출력 (재귀)"""
    for b in blocks:
        line = NotionAPI.block_to_text(b)
        if line.strip():
            print(" " * indent + line)
        # children 재귀 출력
        if "children" in b and b["children"]:
            render_blocks(b["children"], indent + 2)


def main():
    notion = NotionAPI(NOTION_API_KEY)

    print("데이터베이스 정보를 가져오는 중...")
    db_info = notion.get_database(NOTION_DATABASE_ID)
    print(
        f"- 제목: {db_info['title'][0]['plain_text'] if db_info.get('title') else '(제목 없음)'}"
    )
    print(f"- 생성일: {db_info['created_time']}")

    print("\n데이터베이스 전체 페이지 쿼리 중...")
    pages = notion.query_database_all(NOTION_DATABASE_ID)
    print(f"총 {len(pages)}개의 페이지를 찾았습니다.")

    for i, page in enumerate(pages, 1):
        print(f"\n=== 페이지 {i} ===")
        page_id = page["id"]
        print(f"ID: {page_id}")
        print(f"생성일: {page['created_time']}")
        print(f"URL: {page['url']}")

        # 제목 등 속성 출력
        props = page["properties"]
        for prop_name, prop in props.items():
            t = prop["type"]
            if t == "title" and prop["title"]:
                print(f"제목: {prop['title'][0]['plain_text']}")
            elif t == "select" and prop["select"]:
                print(f"{prop_name}: {prop['select']['name']}")
            elif t == "multi_select" and prop["multi_select"]:
                print(f"{prop_name}: {[v['name'] for v in prop['multi_select']]}")
            elif t == "date" and prop["date"]:
                print(
                    f"{prop_name}: {prop['date']['start']} ~ {prop['date'].get('end')}"
                )

        # ------- 내용 가져오기 (트리 형태, 깊이까지 전부) -------
        print("\n[본문]")
        blocks_tree = notion.get_block_tree(page_id)  # 깊이 있는 전체 내용
        if blocks_tree:
            render_blocks(blocks_tree)
        else:
            print("(내용 없음)")


if __name__ == "__main__":
    main()